# Numerical Simulation of Electrochemical Methods

This script is adapted from "Demystifying Mathematical Modelling of Electrochemical Systems" by Stephens and Mauzeroll. The code provided in the SI of this paper is written in Matlab, and the work below adapts it into a functioning pythong script. 

## Linear Sweep Voltammetry 

This code specifically applies to a linear sweep voltamogram for the reduction: Fe(CN)6^3- + e^- -> Fe(CN)6^3-. Many of the other parameters have also been specified by the problem of interest. These paramerts may be altered to represent different redox systems, or even different electrochemical methods. 

In [4]:
import numpy as np 
import matplotlib.pyplot as pyplot
import scipy

### Simulation Input

In [5]:
# Constants
F = 96485 # Faraday’s constant [C/mol]
R = 8.314 # Gas constant [(C*V)/(mol*K)]
T = 298 # Temperature [K]

#Electrochemical parameters 
# Potential waveform
E_start = 0 # Start potential [V] {default: 0}
E_end = -0.8; # End potential [V] {default: -0.8}
E_0 = -0.4 #Standard potential [V] {default: -0.4}
nu = 10 # Scan rate [mV/s] {default: 10}


# Redox mediator
D = 1E-6 # Diffusion coefficient [cm^2/s] {default: 1E-6}
c_ox = 1 # Bulk concentration of oxidized species [M] {default: 1}
c_red = 1E-3 # Bulk concentration of reduced species [M] {default: 1E-3}
n = 1 # Number of electrons transferred during reaction {default: 1}
alpha = 0.5 # Transfer coefficient (unitless) {default: 0.5}

# Electrode
A = 1 # Electrode area [cm^2] {default: 1}
k_0 = 1E-1  # Heterogeneous rate constant [m/s] {default: 1E-1}

# Finite difference parameters 
npts_x = 100 # Number of mesh points {default: 100}
npts_t = 500 # Number of time steps {default: 500}

### Time 0 - Initial Setup and Time Step

In [38]:
# Discretize x
total_x = 0.1 # Maximum x value to simulate [cm]
del_x = total_x/npts_x # x interval
#x = [0:del_x:total_x]’ # Discretized x

# Discretize t
total_t = abs(E_end - E_start)/(nu/1000) # Maximum t value to simulate (1 LSV)
del_t = total_t/npts_t # Time interval
#t = [0:del_t:total_t]’ % Discretized t

# Set uniform concentration along discretized x
cox_x = np.tile(c_ox, [npts_x,npts_t])
cred_x = np.tile(c_red, [npts_x,npts_t])

# Setup empty matrices for time-dependent quantities that will be filled in as the simulation progresses
E_t = np.zeros((npts_t,1))
kred_t = np.zeros((npts_t,1))
i_t = np.zeros((npts_t,1))

# Calculate initial potential
E_curr = E_start # E_curr = Current potential (single value)
E_t[0] = E_curr # E_t = Potential waveform (matrix of values)


# Calculate formal potential according to the Nernst equation
E_eq = E_0 - ((R*T)/(n*F) * np.log(c_ox/c_red))

# Calculate initial rate constant according to the Butler-Volmer equation
eta = E_curr - E_eq # Overpotential (V)
kred_curr = k_0*np.exp((-alpha*n*F*eta)/(R*T))

# Concentration in box 0 updated due to chemical reaction
cred_x[0,0] = (cred_x[0] + (del_t*(kred_curr * cox_x[0,0])))[0] # Equation 17
cox_x[0,0] = (cox_x[0] - (del_t*(kred_curr * cox_x[0,0])))[0] # Equation 20

